In [114]:
import torch
from torch import nn
# import torchvision.transforms as T
# from torchsummary import summary
from matplotlib import pyplot as plt
import h5py
import numpy as np
import xarray as xr
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
# # import torch
# import torch.nn.functional as F
# from torch_geometric_temporal.nn.recurrent import GConvGRU

# class RecurrentGCN(torch.nn.Module):

#     def __init__(self, node_features, num_classes):
#         super(RecurrentGCN, self).__init__()
#         self.recurrent_1 = GConvGRU(node_features, 32, 5)
#         self.recurrent_2 = GConvGRU(32, 16, 5)
#         self.linear = torch.nn.Linear(16, num_classes)

#     def forward(self, x, edge_index, edge_weight):
#         x = self.recurrent_1(x, edge_index, edge_weight)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.recurrent_2(x, edge_index, edge_weight)
#         x = F.relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.linear(x)
#         return F.log_softmax(x, dim=1)

In [3]:
class CNN(nn.Module):
    def _build_vgg_block(self, num_conv_layers, in_channels, out_channels, kernel_size=3, batchnorm=True, activation=nn.ReLU, maxpool=True):
        layers = []
        for i in range(num_conv_layers):
            if i == 0:
                num_channels_in = in_channels
            else:
                num_channels_in = out_channels
            
            layers.append(nn.Conv2d(in_channels=num_channels_in, out_channels=out_channels, kernel_size=kernel_size))
            if batchnorm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(activation())
        
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=2))
        return nn.Sequential(*layers) # this in python is used to unroll the list (**dictionary to unroll a disctionary)

    def __init__(self, num_classes=10, in_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            self._build_vgg_block(2, in_channels, 16, activation=nn.SiLU),
            self._build_vgg_block(2, 16, 32, activation=nn.SiLU)
        )
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, num_classes)
        )

    def forward(self, X):
        return self.classifier(self.avgpool(self.features(X)))

In [4]:
class CNN(nn.Module):
    def __init__(self, input_size):
        super(CNN, self).__init__()
        self.model= nn.Sequential(
            
            nn.Conv3d(input_size, 25, kernel_size=(48,5,5), padding=(0,2,2)), #Conv Layer 1    
            nn.ReLU(),
            # nn.MaxPool3d((1, 1, 2)),
     
            # nn.Conv3d(25, 25, kernel_size=(1,1,1), padding=(0,0,0)),    #Conv Layer 2 
            # nn.ReLU(),
            # nn.MaxPool3d((0, 0, 1)),
               
            # nn.Flatten(),                   #Flatten
            # nn.Linear(3, 1),        #Linear 1
            # nn.ReLU(),                      #Relu
            # nn.BatchNorm1d(128),            #BatchNorm1d
            # nn.Dropout(P=0.15),             #Dropout
            # nn.Linear(48, output_size)               #Linear 2
            )

    def forward(self, X):
        # Set 1
        out = self.model(X)
        return out

In [5]:
def train_epoch(model, inputs, targets, optimizer=torch.optim.Adam, loss_fn=nn.MSELoss, lr=0.001, weight_decay=0.001):
    optimizer=optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_list = []
    optimizer.zero_grad()
    y_pred = model(inputs)
    loss = loss_fn(y_pred, targets.type(torch.float32))
    loss_list.append(loss)
    loss.backward()
    optimizer.step()

def train_model(model, inputs, targets, num_epochs):
    model.train()
    for e in range(num_epochs):
        train_epoch(model, inputs, targets)

In [11]:
inputs = torch.zeros((1, 25, 48, 5, 5)) # period_of_influence*2, lat_dim, lon_dim, n_levels*n_variables + 1
# input.flatten(start_dim=2).shape
targets =torch.zeros((25, 12, 9))

In [12]:
model = CNN(input_size=25)
# train_model(model, inputs, targets, num_epochs=10)

o = model(inputs)
o.shape

torch.Size([1, 25, 1, 12, 9])

In [13]:
o.squeeze().shape

torch.Size([25, 12, 9])

#### Filter Dimensions

A 3D CNN filter has 4 dimensions: [channels, height, width, depth].

#### Overall Input Dimensions

A 3D CNN has 5 dimensional input: [batch_size, channels, height, width, depth]

In [14]:
input_file = '/m100_work/ICT22_ESP_0/vblasone/SLICED_AND_PREPROCESSED/data_slice_5.hdf5'

In [109]:
with h5py.File(input_file, 'r') as f:
    n_lat, n_lon = f['output'].shape[2:4]
    n_samples = f['output'].shape[0]
    lat_idxs = np.random.permutation(np.arange(2,n_lat-2))
    lon_idxs = np.random.permutation(np.arange(2,n_lon-2))
#     print(f['output'].shape)
    for lat_idx in lat_idxs:
        for lon_idx in lon_idxs:
            print(lat_idx, lon_idx)
            input_data = torch.tensor(f['output'][:,:,lat_idx-2:lat_idx+3,lon_idx-2:lon_idx+3,:].reshape(n_samples,25,48,5,5)).to(device)
            print(input_data.shape)
            break
        break

3 5
torch.Size([5843, 25, 48, 5, 5])


In [112]:
model.to(device)
model(input_data.type(torch.float32)).shape

torch.Size([5843, 25, 1, 5, 5])

In [117]:
target = xr.open_dataset('/m100_work/ICT22_ESP_0/vblasone/GRIPHO/gripho_slice_5.nc')

In [124]:
target['pr']

<xarray.DataArray 'pr' (time: 140256, y: 102, x: 58)>
[829754496 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01T00:22:30 ... 2016-12-31T23:00:00
    lon      (y, x) float32 8.034 8.068 8.103 8.137 ... 9.881 9.916 9.952 9.988
    lat      (y, x) float32 38.74 38.74 38.74 38.75 ... 41.51 41.51 41.51 41.51
  * x        (x) float64 4.149e+06 4.152e+06 4.155e+06 ... 4.317e+06 4.32e+06
  * y        (y) float64 1.742e+06 1.745e+06 1.748e+06 ... 2.042e+06 2.045e+06
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          mm h-1
    grid_mapping:   crs